In [4]:
# !pip install transformers datasets peft accelerate bitsandbytes

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
import os

# Disable Weights & Biases logging
os.environ["WANDB_DISABLED"] = "true"

# 1. Load model + tokenizer
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="right")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name)

# 2. Apply LoRA config
peft_config = LoraConfig(task_type="CAUSAL_LM", r=8, lora_alpha=16, lora_dropout=0.1)
model = get_peft_model(model, peft_config)

# 3. Tiny dataset
dataset = load_dataset("yelp_polarity", split="train[:200]").map(
    lambda x: tokenizer(x["text"], truncation=True, padding="max_length", max_length=64),
    batched=True
)

# 4. Training
args = TrainingArguments(
    output_dir="out",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    report_to="none"   # <- avoids W&B
)
trainer = Trainer(model=model, args=args, train_dataset=dataset)
trainer.train()


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2174: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


ValueError: Expected input batch_size (256) to match target batch_size (4).